In [0]:
%pylab inline

In [0]:
import meteo
import scipy.optimize as opt
from IPython.html.widgets import interact
import seaborn as sns
sns.set_style('white')

In [0]:
N = 128
qmin = 0.0
qmax = 1.0

g = 0.02
delta = 0.1
gamma = 2.0
c = 250.0
d = 100.0
beta = 3.0
nu = 1.0

Wmin = 0.0
Wmax = gamma*c*delta**(gamma-1)
Wfunc = lambda W: delta*W-1.0-(gamma-1)*c*(W/(gamma*c))**(gamma/(gamma-1))
#Wbar = opt.brentq(Wfunc,Wmin,Wmax)
Wbar = opt.fsolve(Wfunc,1.0)[0]
x0 = (Wbar/(c*gamma))**(1/(gamma-1))
print 'Wbar = {}, Werr = {}, x0 = {}, R0 = {}'.format(Wbar,Wfunc(Wbar),x0,c*x0**gamma)

qgrid = np.linspace(qmin,qmax,N)
dqgrid = np.diff(qgrid)
fgrid = nu*qgrid**(nu-1) # qgrid*(1-qgrid)**(beta-1)*beta*(beta+1)
Fgrid = np.r_[0.0,np.cumsum(fgrid[:-1]*dqgrid)]
Wgrid = Wbar*qgrid
Wpgrid = Wbar*np.ones_like(qgrid)
Zpgrid = -Wgrid*fgrid
Zgrid = np.r_[0.0,np.cumsum(-(Zpgrid[1:]*dqgrid)[::-1])][::-1]
xgrid = (Wbar/(gamma*c))**(1/(gamma-1))*np.ones_like(qgrid)
ygrid = (Zgrid/(gamma*d))**(1/(gamma-1))*np.ones_like(qgrid)

constants = {'g': g,'delta': delta,'gamma': gamma,'c': c,'d': d,'Wbar': Wbar,'x0': x0, 'F': Fgrid, 'f': fgrid}

In [0]:
mod_eq = meteo.Model('models/frontier_equil.json',constants=constants,sparse=True)

In [0]:
(t_path_eq,par_path_eq,var_path_eq) = mod_eq.homotopy_elev({"alpha":0.0,"g":0.0},{"alpha":1.0,"g":g},
                                       {"Wbar":Wbar,"W":[Wgrid,Wpgrid],"Z":[Zgrid,Zpgrid],"x":xgrid,"y":ygrid},
                                       solve=True,output=True,delt=0.1,max_step=1000,max_newton=50)

t_path_good_eq = t_path_eq[::2]
par_path_good_eq = par_path_eq[::2]
var_path_good_eq = var_path_eq[::2]

parf_dict_eq = mod_eq.array_to_dict(par_path_good_eq[-1],mod_eq.par_info,mod_eq.par_sizes)
varf_dict_eq = mod_eq.array_to_dict(var_path_good_eq[-1],mod_eq.var_info,mod_eq.var_sizes)

In [0]:
mod_sp = meteo.Model('models/frontier_socplan.json',constants=constants,sparse=True)

In [0]:
(t_path_sp,par_path_sp,var_path_sp) = mod_sp.homotopy_elev({"alpha":0.0,"g":0.0},{"alpha":1.0,"g":g},
                                       {"Wbar":Wbar,"W":[Wgrid,Wpgrid],"Z":[Zgrid,Zpgrid],"x":xgrid,"y":ygrid},
                                       solve=True,output=True,delt=0.1,max_step=1000,max_newton=20)

t_path_good_sp = t_path_sp[::2]
par_path_good_sp = par_path_sp[::2]
var_path_good_sp = var_path_sp[::2]

parf_dict_sp = mod_sp.array_to_dict(par_path_good_sp[-1],mod_sp.par_info,mod_sp.par_sizes)
varf_dict_sp = mod_sp.array_to_dict(var_path_good_sp[-1],mod_sp.var_info,mod_sp.var_sizes)

In [0]:
# compare decentralized equilibrium and social planner's optimum
#gy_fact = (1/(nu+1))*(qgrid**(nu+1)-(nu+1)*qgrid+nu)/qgrid

x_eq = varf_dict_eq['x']
y_eq = varf_dict_eq['y']
#g_eq = x_eq + y_eq*gy_fact
R_eq = c*x_eq**gamma + d*y_eq**gamma

x_sp = varf_dict_sp['x']
y_sp = varf_dict_sp['y']
#g_sp = x_sp + y_sp*gy_fact
R_sp = c*x_sp**gamma + d*y_sp**gamma

(fig,(ax1,ax2,ax3)) = subplots(1,3,figsize=(15,5))
ax1.plot(qgrid,x_eq,qgrid,x_sp); sns.despine(ax=ax1);
ax2.plot(qgrid,y_eq,qgrid,y_sp); sns.despine(ax=ax2);
ax3.plot(qgrid,R_eq,qgrid,R_sp); sns.despine(ax=ax3);

In [0]:
mod_po = meteo.Model('models/frontier_privopt.json',constants=meteo.merge(constants,{'x': x_sp,'y': y_sp}),sparse=True)

In [0]:
(t_path_po,par_path_po,var_path_po) = mod_po.homotopy_elev({"alpha":0.0,"g":0.0},{"alpha":1.0,"g":g},
                                       {"Wbar":Wbar,"x0":x0,"W":[Wgrid,Wpgrid],"Z":[Zgrid,Zpgrid]},
                                       solve=True,output=True,delt=0.1,max_step=1000,max_newton=20)

t_path_good_po = t_path_po[::2]
par_path_good_po = par_path_po[::2]
var_path_good_po = var_path_po[::2]

parf_dict_po = mod_po.array_to_dict(par_path_good_po[-1],mod_po.par_info,mod_po.par_sizes)
varf_dict_po = mod_po.array_to_dict(var_path_good_po[-1],mod_po.var_info,mod_po.var_sizes)

In [0]:
(W_PO,dW_PO) = varf_dict_po['W']
(Z_PO,dZ_PO) = varf_dict_po['Z']

(W_SO,dW_SO) = varf_dict_sp['W']
(Z_SO,dZ_SO) = varf_dict_sp['Z']

sx = 1.0 - dW_PO/dW_SO
sy = 1.0 - Z_PO/(Z_SO-(1-Fgrid)*W_SO)

Rx_sp = c*x_sp**gamma
Ry_sp = d*y_sp**gamma

(fig,(ax1,ax2)) = subplots(1,2,figsize=(12,5))
ax1.plot(qgrid,-sx/(1-sx)); ax1.set_ylim(0.0,0.2); sns.despine(ax=ax1);
ax2.plot(qgrid,-sy/(1-sy)); ax2.set_ylim(0.0,1.0); sns.despine(ax=ax2);